In [379]:
import pandas as pd
import requests
import datetime as dt
import time
import re

from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

import spacy
import spacy.cli

import contractions
import unicodedata
import string

In [82]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
#pd.set_option('max_colwidth',800)

In [84]:
parent_askscience=pd.read_csv('./datasets/askscience/submissions.csv')

In [222]:
parent_askengineers=pd.read_csv('./datasets/askengineers/submissions.csv',nrows=5074)

In [224]:
parent_askscience.shape, parent_askengineers.shape

((5074, 102), (5074, 80))

In [225]:
parent_askscience.columns

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_background_color', 'author_flair_css_class',
       'author_flair_richtext', 'author_flair_text', 'author_flair_text_color',
       'author_flair_type', 'author_fullname',
       ...
       'can_gild', 'hidden', 'media', 'quarantine', 'secure_media',
       'subreddit_name_prefixed', 'rte_mode', 'author_id', 'brand_safe',
       'previous_visits'],
      dtype='object', length=102)

In [226]:
#picking some features that might be interesting to explore
features=['created_utc','id','author','num_comments','score','selftext','subreddit','title']

In [513]:
askscience_df=parent_askscience[features].copy()
askengineers_df=parent_askengineers[features].copy()
askscience_df.shape, askengineers_df.shape

((5074, 8), (5074, 8))

In [331]:
askscience_df['subreddit'].value_counts()

askscience    5074
Name: subreddit, dtype: int64

In [332]:
askengineers_df['subreddit'].value_counts()

AskEngineers    5074
Name: subreddit, dtype: int64

In [333]:
askscience_df.head()

,created_utc,id,author,num_comments,score,selftext,subreddit,title
0,1663962461,xm7a6n,bmarcus128,0,1,"My typical body temperature hovers around 97.8 F. I'm recovering from COVID, and my temperature is currently about 98.8 F. Do I have a fever? (In other words, is a fever an elevated temperature compared to an individual's baseline or is it a temperature above a certain number?)",askscience,Does 98.6 Count as a Fever if Your Average Body Temp Trends Low?
1,1663947109,xm0pl0,ChrisGnam,0,1,"Electrons are extraordinarily simple, composed of just a few quantum properties like charge, spin, and mass. Why would we not expect every single Electron to have these same identical properties? Especially since these properties are quantized (i.e., you cannot have a spin that is not a half integer value, or a charge that is not an integer multiple of a fundamental charge).\n\nWas the original postulate even more light hearted than I thought (even if it did provide insight to Feynman)? Or is there actually something interesting to be learned by realizing all electrons are the same?",askscience,"The ""one-Electron universe"" was somewhat humorously proposed to explain why every Electron is identical. But why would that need explaining? Why should we expect electrons to not be identical?"
2,1663930837,xludsa,AskScienceModerator,0,1,"**What are you doing to get clean air?**\n\nAs the fear of airborne illnesses has heightened, people have understandably become much more concerned about potentially contagious strangers and are choosing to go where high air quality standards are maintained. Conventional approaches have been focused on making filters tighter, but filter media can only be made so small before becoming difficult to move air effectively or efficiently. Reducing nanoparticles by agglomerating them into larger, aggregated clusters is the key. Larger particles are more easily mitigated and fall to the ground more quickly. Atmofizer is leading 4th generation in air technology, with patented agglomeration tech using acoustic pressure waves to reduce airborne nanoscopic particles-without using filters and savin...",askscience,"AskScience AMA Series: Hi! We are Stephanie Walton and Whit Pepper, scientist and chief commercial officer of clean-air company ATMOFIZER with patented technology designed to reduce indoor ultra-fine particles and deactivate airborne viruses/bacteria without using a filter. Ask us Anything!"
3,1663758037,xk1pns,AskScienceModerator,0,1,"Join our expert panel to discuss this year's #PeerReviewWeek22 theme #ResearchIntegrity: creating and supporting trust in research.\n\nThe Royal Society is hosting a live session on 21st September to enable our community all over the world to interact with industry experts. Simply reply to this post with your peer review questions following the theme of #ResearchIntegrity before or during the event and we'll answer them live, giving you a diverse range of answers.\n\nWe'll be on at 3pm BST (11 AM ET), ask us anything!\n\n**Panellist biographies**\n\n+ **Panellist - Professor John Hutchinson**, Professor of Evolutionary Biomechanics, Royal Veterinary College, University of London\n + [Professor John Hutchinson](https://www.rvc.ac.uk/about/our-people/john-hutchinson) is a Professor of E...",askscience,AskScience AMA Series: We're excited to bring you industry experts from the Royal Society for Peer Review Week 2022. Join our experts who will be answering all your questions around the theme 'Research Integrity: Creating and supporting trust in research'. Ask us anything! All welcome.
4,1663420203,xgm3s1,PlacatedPlatypus,0,1,"I recently read that the oldest dog ever lived to be more than 29 years old. Even by the most conservative ""dog age to human age"" calculations, this seems to be around 140 years in human age. This is significantly higher than the oldest human lifespan ever recorded, which led me to wonder if the translation breaks down as dogs get older. \n\nIt's already known to be 

In [334]:
askscience_df.isnull().sum()

created_utc     0
id              0
author          0
num_comments    0
score           0
selftext        0
subreddit       0
title           0
dtype: int64

In [335]:
askengineers_df.isnull().sum()

created_utc     0
id              0
author          0
num_comments    0
score           0
selftext        0
subreddit       0
title           0
dtype: int64

In [336]:
askscience_df.dtypes

created_utc      int64
id              object
author          object
num_comments     int64
score            int64
selftext        object
subreddit       object
title           object
dtype: object

In [337]:
askengineers_df.dtypes

created_utc      int64
id              object
author          object
num_comments     int64
score            int64
selftext        object
subreddit       object
title           object
dtype: object

In [514]:
#checking for duplicate posts
askscience_df['id'].duplicated().any(), askengineers_df['id'].duplicated().any()

(False, False)

In [515]:
#help from https://stackoverflow.com/questions/61295381/create-new-dataframe-column-from-the-values-of-2-other-columns
askscience_df['title_st']=(askscience_df['title']+' '+ askscience_df['selftext'])
askengineers_df['title_st']=(askengineers_df['title']+' '+ askengineers_df['selftext'])

In [512]:
askengineers_df.head(2);

In [ ]:
#removing decimal between letters (want to keep decimal between )
#st['title_st']=st['title_st'].map(lambda x: re.sub(r'(\.\s)|(\.\w)|(\w\.)',' ', x))

In [456]:
#Help from https://medium.com/mlearning-ai/nlp-a-comprehensive-guide-to-text-cleaning-and-preprocessing-63f364febfc5


def text_cleaning(df):
    #remove url 
    df['title_st']=df['title_st'].map(lambda x: re.sub(r'(https?:\S*)|(www\.\S*)','', x))
    df['title_st']=df['title_st'].map(lambda x: re.sub(r'&amp;#x200B;','', x))
    df['title_st']=df['title_st'].map(lambda x: re.sub(r'E&amp;M','', x))
    df['title_st']=df['title_st'].map(lambda x: re.sub(r'R&amp;D','', x))
    df['title_st']=df['title_st'].map(lambda x: re.sub(r'1&amp;2','', x))
    
    #lower case
    df['title_st']=df['title_st'].map(lambda x: x.lower())
    
    #expanding contractions
    df['title_st']=df['title_st'].map(lambda x: ' '.join([contractions.fix(word) for word in x.split()]))
    
    #standardizing accent characters
    df['title_st']=df['title_st'].map(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore'))
        
    # remove new line, lab, carriage return
    df['title_st']=df['title_st'].map(lambda x: re.sub(r'(\n)|(\t)|(\r)','', x))
    
    # remove_mentions_and_tags
    df['title_st']=df['title_st'].map(lambda x: re.sub(r'@\S*','', x))
    df['title_st']=df['title_st'].map(lambda x: re.sub(r'#\S*','', x))
   
    #removing text related to images
    df['title_st']=df['title_st'].map(lambda x: re.sub(r'\[.+[ImgBBibb].+\]','', x))
         
    
    #removing decimal between letters (want to keep decimal between )
    #df['title_st']=df['title_st'].map(lambda x: re.sub(r'(\.\s)|(\.\w)|(\w\.)',' ', x))


    #defining positive and negative numbers
    df['title_st']=df['title_st'].map(lambda x: re.sub(r'(\+\s?\d)',' positivenum ', x))
    df['title_st']=df['title_st'].map(lambda x: re.sub(r'(\-\s?\d)',' negativenum ', x))

    #defining euqation
    df['title_st']=df['title_st'].map(lambda x: re.sub(r'\S+=\S+',' equation ', x))
    
    #defining float numbers
    df['title_st']=df['title_st'].map(lambda x: re.sub(r'\d\.\d',' floatnum ', x))
    
    #defining dimensions
    df['title_st']=df['title_st'].map(lambda x: re.sub(r'[\d\w]+\s*[xX]\s*[\d\w]+\s*[xX]\s*[\d\w]+',' dimension ', x))
    df['title_st']=df['title_st'].map(lambda x: re.sub(r'[\d\w]+\s*[xX]\s*[\d\w]+',' dimension ', x))
    
   # defining percentage
    df['title_st']=df['title_st'].map(lambda x: re.sub(r'[\d]+\s*\%',' percentage ', x))
    
    #removing digits
    df['title_st']=df['title_st'].map(lambda x: re.sub(r'[^a-zA-z.,!?/:;\"\'\s]','', x))
    
     # remove special characters: #, [, ], (, ), \,_,/,*,$,@
    df['title_st']=df['title_st'].map(lambda x: re.sub(r'(#)|(\[)|(\])|(\()|(\))|(\\)|(_)|(\/)|(\*)|(\$)|(@)','', x))
    
    #remove punctuatons
    df['title_st']=df['title_st'].map(lambda x: ''.join([char for char in x if char not in string.punctuation]))
    
    return (df)

In [516]:
askengineers_df_clean=text_cleaning(askengineers_df)

In [517]:
askscience_df_clean=text_cleaning(askscience_df)

In [519]:
#concatenating askscience and askengineers dataframe

ask_df=pd.concat([askscience_df_clean,askengineers_df_clean])

In [521]:
ask_df.shape

(10148, 9)

In [525]:
ask_df['subreddit'].value_counts()

askscience      5074
AskEngineers    5074
Name: subreddit, dtype: int64

In [526]:
ask_df['subreddit']=ask_df['subreddit'].map({'askscience':1, 'AskEngineers':0})

In [527]:
ask_df['subreddit'].value_counts()

1    5074
0    5074
Name: subreddit, dtype: int64

In [533]:
ask_df.head(2);

In [535]:
#creating a column for length of title + text
ask_df['title_st_length']=ask_df['title_st'].map(lambda x: len(x))

In [536]:
#creating a column for word counts of title + text
ask_df['title_st_word_count']=ask_df['title_st'].map(lambda x: len(x.split(" ")))

In [539]:
ask_df.head(2);

In [132]:
nlp=spacy.load('en_core_web_md')

In [529]:
ask_df['title_st_lemma']=ask_df['title_st'].map(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))

In [530]:
ask_df['title_st_lemma'][:2]

0                                                                                                                                                                                                                                                                                                                                                                                                                           do   floatnum   count as a fever if your average body temp trend low my typical body temperature hover around   floatnum   f I be recover from covid and my temperature be currently about   floatnum   f do I have a fever in other word be a fever an elevated temperature compare to an individual baseline or be it a temperature above a certain number
1    the oneelectron universe be somewhat humorously propose to   dimension   why every electron be identical but why would that need   dimension   why should we   dimension   electron to not be identical electron be   dimension   simple c

In [511]:
askengineers_df_clean['title_st'].sample()

2724    advise to design a braking system I be a nd year undergraduate and have a project to design a braking system for the new hyperloop club at our college my major be computational mechanic so I do have basic knowledge in solid mechanic and stuff but that be only theoretical I have not design a product physically until now I have learn basic electromagnetic theory via griffith and gain some skillset in solidwork and comsol since the braking system we be think of do would be an integrate magnetic eddy current   friction brake   what tool and theory should I have rigorous knowledge in early on to make this happen like fem etc    how should my progress look like that be what should be my   dimension   step and so on   any other general tip   recommendation I apologise if the question be too g...
Name: title_st, dtype: object

In [490]:
doc=st.iloc[1730,0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [135]:
doc

'I tried to ask ELI5 to help me understand Sabrina Pasterski’s discoveries, as they’re a bit over my head, but was told this might be a more reliable place for information. I’m specifically interested in the  Pasterski-Strominger-Zhiboedov Triangle and it’s potential importance, relevance, and application. I know it ties into black holes, the difference between 2, 3, and 4 dimensions, and it sounds like it lends support to theories put forth by Weinberg. But it’s so far more advanced than I can grasp right now. Even the stuff I think I understand could very well be wrong. Would anybody care to use some layman’s terms to help me grasp these concepts Thank you!'

In [137]:
ex = nlp(doc)
print(ex.text)

I tried to ask ELI5 to help me understand Sabrina Pasterski’s discoveries, as they’re a bit over my head, but was told this might be a more reliable place for information. I’m specifically interested in the  Pasterski-Strominger-Zhiboedov Triangle and it’s potential importance, relevance, and application. I know it ties into black holes, the difference between 2, 3, and 4 dimensions, and it sounds like it lends support to theories put forth by Weinberg. But it’s so far more advanced than I can grasp right now. Even the stuff I think I understand could very well be wrong. Would anybody care to use some layman’s terms to help me grasp these concepts Thank you!


In [ ]:
ex.